In [ ]:
import requests
import json

ollama_api_url = 'http://localhost:11434/api/chat'

def stream_ollama_response(model, prompt):
    # 為 /api/chat 端點準備請求的 payload
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": True
    }

    try:
        with requests.post(ollama_api_url, json=payload, stream=True) as response:
            # 檢查請求是否成功
            response.raise_for_status()
            
            # 逐行迭代回應內容
            for line in response.iter_lines():
                if line:
                    try:
                        # 每一行都是一個獨立的 JSON 物件
                        json_data = json.loads(line.decode('utf-8'))
                        
                        # 從 'message' 物件中提取 'content'
                        content_chunk = json_data.get('message', {}).get('content', '')
                        
                        # 使用 yield 回傳每一塊內容
                        yield content_chunk
                        
                        # 最後一個 JSON 物件會有一個 'done' 欄位為 true
                        if json_data.get('done'):
                            break
                    except json.JSONDecodeError:
                        print(f"\n[警告] 無法解析的 JSON 行: {line}")
                        
    except requests.exceptions.RequestException as e:
        print(f"\n[錯誤] 請求失敗: {e}")

# --- 如何使用 ---

model_to_use = "gemma3:4b"
user_prompt = "中國迅速侵蝕美國全球 AI 競賽領導地位這件事，請提供詳細的分析。"

print(f"--- 正在從 {model_to_use} 接收串流式應答 ---")

# for 迴圈會自動處理 generator，並在收到每個區塊時立即印出
full_response = ""
for chunk in stream_ollama_response(model_to_use, user_prompt):
    print(chunk, end='', flush=True)
    full_response += chunk

print("\n\n--- 串流結束 ---")
# print("\n完整的回應:\n", full_response) # 您也可以在最後檢視完整回應


In [8]:
import requests
import json

ollama_api_url = 'http://localhost:1234/ollama/chat'

def stream_ollama_response(prompt):
    try:
        with requests.post(ollama_api_url, json={ "prompt": prompt }, stream=True) as response:
            response.raise_for_status()

            for line in response.iter_lines():
                if line:
                    try:
                        json_data = json.loads(line.decode('utf-8'))
                        
                        content_chunk = json_data.get('message', {}).get('content', '')
                        
                        yield content_chunk
                        
                        if json_data.get('done'):
                            break
                    except json.JSONDecodeError:
                        print(f"\n[警告] 無法解析的 JSON 行: {line}")
                        
    except requests.exceptions.RequestException as e:
        print(f"\n[錯誤] 請求失敗: {e}")

def ask_ollama(prompt):
	full_string = ""
	for chunk in stream_ollama_response(prompt):
		print(chunk, end='', flush=True)
		full_string += chunk

In [10]:
ask_ollama('python的yield怎麼用？')

`yield` 是 Python 中一个非常重要的关键字，它允许你创建 **生成器 (generator)**。 生成器函数与普通函数的主要区别在于，普通函数在被调用时会立即执行并返回值，而生成器函数在被调用时不会执行任何代码，而是返回一个生成器对象。  当需要用到生成器中的数据时，调用生成器对象的 `next()` 方法，它会逐个产生值，直到生成器耗尽。

**理解 `yield` 的作用**

* **暂停执行:**  `yield` 语句会暂停当前函数的执行，并将当前的值返回给调用者。
* **恢复执行:**  当调用者再次调用生成器对象的 `next()` 方法时，生成器函数会从 `yield` 语句之后继续执行，直到遇到下一个 `yield` 语句。
* **状态保存:**  `yield` 语句在暂停执行时会保存函数的状态（包括变量的值、栈指针等），以便在下次执行时从上次停下的位置继续执行。

**使用 `yield` 创建生成器**

1. **定义生成器函数：**  生成器函数必须定义在一个普通函数中，并使用 `def` 关键字进行定义。
2. **使用 `yield` 语句：** 在函数体中使用 `yield` 语句来产生值。
3. **调用生成器函数：** 调用生成器函数不会执行任何代码，而是返回一个生成器对象。

**举例说明**

**1. 简单的生成器：**

```python
def my_generator():
    yield 1
    yield 2
    yield 3

# 创建生成器对象
gen = my_generator()

# 使用 next() 方法获取值
print(next(gen))  # 输出: 1
print(next(gen))  # 输出: 2
print(next(gen))  # 输出: 3

try:
    print(next(gen))
except StopIteration:
    print("生成器已耗尽")
```

**2.  生成斐波那契数列：**

```python
def fibonacci_generator(max_value):
    a, b = 0, 1
    while a <= max_value:
        yield a
     